In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns

In [ ]:
from src.preprocessing import Preprocessing

In [4]:
!pip3 install gensim
!pip3 install pyLDAvis
# !conda init bash

In [3]:
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample

In [4]:
df = pd.read_csv('../cleaned_tweet_data.csv')

In [ ]:
preprocess = Preprocessing(df)

In [5]:
df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favourite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,2022-04-22 22:17:05+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @NorthstarCharts: The 10-year yield is tell...,0.16,0.540000,en,188.0,43.0,davideiacovozzi,18,18,NaN,"[{'text': 'gold', 'indices': [116, 121]}, {'te...","[{'screen_name': 'NorthstarCharts', 'name': 'N...",NaN
1,2022-04-22 13:44:53+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @MichaelAArouet: German 10y mortgage rate w...,0.15,0.175000,en,179.0,32.0,davideiacovozzi,18,18,NaN,[],"[{'screen_name': 'MichaelAArouet', 'name': 'Mi...",NaN
2,2022-04-22 06:10:34+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @goldseek: When? https://t.co/kO2FfHKaZg,0.00,0.000000,en,193.0,26.0,davideiacovozzi,18,18,False,[],"[{'screen_name': 'goldseek', 'name': 'Peter ⚒ ...",NaN
3,2022-04-21 17:22:09+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @charliebilello: The 30-year mortgage rate ...,0.00,0.183333,en,620.0,213.0,davideiacovozzi,18,18,NaN,[],"[{'screen_name': 'charliebilello', 'name': 'Ch...",NaN
4,2022-04-21 10:32:26+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @biancoresearch: Rates rise until something...,-0.40,0.400000,en,1787.0,417.0,davideiacovozzi,18,18,False,[],"[{'screen_name': 'biancoresearch', 'name': 'Ji...",NaN


In [6]:
df.shape

(15076, 15)

In [29]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

# clean text 
df = preprocess.cleantext('original_text', 'clean_text')

# Stem words
df = preprocess.stem('clean_text')

In [11]:
df['original_text'][0]

"RT @NorthstarCharts: The 10-year yield is telling us that there's a high risk of something 'breaking' in the system #gold #silver #crypto #…"

In [13]:
df['clean_text'][0]

'rt northstarchart the yield tell us there high risk someth break system gold silver crypto'

In [8]:
df.to_csv('../clean_tweet.csv')

In [14]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/jedi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jedi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# Used stemmer instead of lemmatizer
cleanKagTweetList = preprocess.filterTweetList(df['clean_text'])

In [18]:
cleanKagTweetList[:5]

[['northstarchart',
  'yield',
  'tell',
  'high',
  'risk',
  'someth',
  'break',
  'system',
  'gold',
  'silver',
  'crypto'],
 ['michaelaarouet',
  'german',
  'mortgag',
  'rate',
  'went',
  'hear',
  'sound',
  'german',
  'real',
  'estat',
  'bubbl',
  'burst'],
 ['goldseek'],
 ['charliebilello',
  'mortgag',
  'rate',
  'rise',
  'highest',
  'level',
  'sinc',
  'last',
  'year',
  'hit',
  'alltim'],
 ['biancoresearch',
  'rate',
  'rise',
  'someth',
  'break',
  'anyth',
  'broken',
  'yet']]

In [19]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=50,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [21]:
"""Create model objects"""
kagDict   = makeDict(cleanKagTweetList)
kagCorpus = makeCorpus(cleanKagTweetList, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

"""Save model objects"""
SaveLoad.save(kagLda,'kaggleLDAmodel')
corpora.MmCorpus.serialize('kaggleCorpus.mm', kagCorpus)
kagDict.save('kaggleDictionary.dict')

In [22]:
kagLda = SaveLoad.load('kaggleLDAmodel')
kagDict = corpora.Dictionary.load('kaggleDictionary.dict')
kagCorpus = corpora.MmCorpus('kaggleCorpus.mm')

In [23]:
!pip3 install pyLDAvis

In [24]:
# import pyLDAvis
import pyLDAvis.gensim_models

In [25]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim_models.prepare(kagLda, kagCorpus, kagDict)

/home/jedi/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/home/jedi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jedi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jedi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jedi/anaconda3/lib/python3.9/site-packages/past/builtins/mis

In [41]:
kagLda.print_topics()

[(46,
  '0.022*"minist" + 0.017*"money" + 0.015*"congress" + 0.013*"amp" + 0.008*"surpris" + 0.008*"print" + 0.008*"say" + 0.007*"gujarat" + 0.007*"today" + 0.007*"come"'),
 (33,
  '0.023*"peopl" + 0.019*"done" + 0.014*"noth" + 0.010*"govern" + 0.010*"say" + 0.010*"listen" + 0.007*"young" + 0.007*"never" + 0.007*"india" + 0.007*"laugh"'),
 (17,
  '0.012*"last" + 0.010*"mask" + 0.010*"check" + 0.010*"care" + 0.009*"easter" + 0.009*"amp" + 0.007*"mandat" + 0.007*"food" + 0.006*"take" + 0.006*"peopl"'),
 (42,
  '0.019*"vaccin" + 0.014*"success" + 0.012*"farmer" + 0.011*"sure" + 0.009*"dont" + 0.009*"video" + 0.008*"struggl" + 0.008*"amp" + 0.008*"quit" + 0.007*"commit"'),
 (9,
  '0.064*"srilanka" + 0.061*"cartoon" + 0.056*"lka" + 0.044*"economiccrisislk" + 0.037*"srilankacrisi" + 0.021*"srilankaprotest" + 0.018*"gohomegota" + 0.012*"policebrut" + 0.011*"namalamaras" + 0.009*"rambukkana"'),
 (22,
  '0.038*"ukrain" + 0.017*"russian" + 0.016*"tax" + 0.014*"ukrainian" + 0.012*"agre" + 0.011*"

In [1]:
ldaViz

NameError: name 'ldaViz' is not defined

In [27]:
def translateLdaIdx(myLdaModel, myLdaViz):
    """Translate lda model topics to match the topics in pyLDAvis visualization"""
    ldaVizIdx = myLdaViz[0].index
    return list(ldaVizIdx)

In [28]:
newIdx = translateLdaIdx(kagLda,ldaViz)


In [29]:
def createDenseMat(myLdaModel,myCorpus,newIdx):
    """Transform corpus to dataframe with topics matching lda visualization"""
    numTopics = myLdaModel.num_topics
    myDense = corpus2dense(myLdaModel[myCorpus],numTopics)
    myDf = pd.DataFrame(myDense)
    mySortedDf = myDf.transpose()
    mySortedDf = myDf.transpose()[newIdx]
    mySortedDf.columns = ['topic' + str(i + 1) for i in range(numTopics)]
    return mySortedDf

In [30]:
kagDf = createDenseMat(kagLda,kagCorpus,newIdx)


In [31]:
def sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Returns an index of tweets surpassing a topic value threshold"""
    myCleanArray = np.array(cleanedTweetList)
    srtIdx = list(mySortedDf[mySortedDf[myTopic]>myTopicThresh].index)
    return srtIdx

def sortTweetsByIdx(cleanedTweetList,srtIdx):
    """Returns sorted tweets as a list based on a defined sort index"""
    myCleanArray = np.array(cleanedTweetList)
    srtTweets = list(myCleanArray[srtIdx])
    return srtTweets

In [33]:
!pip3 install numpy
import numpy as np

In [34]:
sortedIdx = sortByTopicToIdx(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.1)


/tmp/ipykernel_21909/43916758.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)


In [36]:
# Word cloud for topic 
def makeWordCloud(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1)
    mySortedTweets = sortTweetsByIdx(cleanedTweetList,sortedIdx)
    filteredWords = ' '.join([' '.join(string) for string in mySortedTweets])
    myTopicCloud = WordCloud(max_font_size=100,scale=8).generate(filteredWords)
    fig = plt.figure(figsize=(10,10), dpi=1600)
    plt.imshow(myTopicCloud)
    plt.axis("off")
    plt.show()

In [37]:
# It can be seen that most tweeted words are related to war in unkraine and economic crisis.
makeWordCloud(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.2)


/tmp/ipykernel_21909/43916758.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)
/tmp/ipykernel_21909/43916758.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)
